# Setup the APR restraints, make the APR windows, and solvate each host-guest complex in each window...

In [13]:
import glob as glob
import os as os
import subprocess as sp
import numpy as np
import shutil as shutil

import parmed as pmd
import pytraj as pt
from paprika.restraints import static_DAT_restraint, DAT_restraint, \
                               create_window_list, amber_restraint_line
from paprika.utils import make_window_dirs
from paprika.tleap import System

In [14]:
complexes = glob.glob('./OA-G*')

First, let's define the dummy atoms that act as anchor particles and three atoms on the host that are distributed around the cavity and will stay relatively rigid during the calculation.

In [15]:
dummy_anchors = [':3', ':4', ':5']
host_anchors  = [':1@C26', ':1@C18', ':1@C8']

Next, we'll define the six "static" restraints that enable the APR calculation by connecting the dummy atoms to the host anchor atoms.

In [16]:
static_restraint_atoms = [[dummy_anchors[0], host_anchors[0]],
                          [dummy_anchors[1], dummy_anchors[0], host_anchors[0]],
                          [dummy_anchors[2], dummy_anchors[1], dummy_anchors[0], host_anchors[0]],
                          [dummy_anchors[0], host_anchors[0], host_anchors[1]],
                          [dummy_anchors[1], dummy_anchors[0], host_anchors[0], host_anchors[1]],
                          [dummy_anchors[0], host_anchors[0], host_anchors[1], host_anchors[2]],
                        ]

static_restraint_distance_fc = 5.0 # kcal/mol-A**2
static_restraint_angle_fc = 100.0  # kcal/mol-rad**2

We will setup a calculation with 15 attach windows and 45 pulling windows.

The true distances for pulling will be offset by the distance between the closest dummy atom and the closest guest atom in the solvated structure.

In [17]:
attach_string = '0.00 0.40 0.80 1.60 2.40 4.00 5.50 8.65 11.80 ' \
                '18.10 24.40 37.00 49.60 74.80 100.00'
attach_fractions = [float(i) / 100 for i in attach_string.split()]

pull_string = '0.00 0.40 0.80 1.20 1.60 2.00 2.40 2.80 3.20 3.60 4.00 ' \
              '4.40 4.80 5.20 5.60 6.00 6.40 6.80 7.20 7.60 8.00 8.40 ' \
              '8.80 9.20 9.60 10.00 10.40 10.80 11.20 11.60 12.00 12.40 ' \
              '12.80 13.20 13.60 14.00 14.40 14.80 15.20 15.60 16.00 16.40 ' \
              '16.80 17.20 17.60 18.00'
                
print(f'There are {windows} windows in this attach-pull calculation.')

There are [15, 46, 0] windows in this attach-pull calculation.


Now we're ready to apply the restraints to each complex.

1. Define guest anchor atoms for `G3` and `G6`.
2. Define the pulling distance for each guest, based on its initial conformation.
3. Initialize the static restraints.
4. Initialize the guest restraints.
5. Make window directories inside `OA-G?-?/AMBER/APR/windows/`.
6. Write a restraint file inside each window.

In [18]:
for complx in complexes:
    print(complx)
    
    # Set the guest anchor atoms
    ############################
    if 'G3' in complx:
        guest_anchors = [':2@C1', ':2@C3']
        traj = pt.load(os.path.join(complx, 'AMBER', 'solvate.rst7'), 
                            os.path.join(complx, 'AMBER', 'solvate.prmtop'))
        
        pull_initial = pt.distance(traj, ' '.join([dummy_anchors[0], guest_anchors[0]]))[0]
        
        print(f'Set pull offset ({pull_initial:1.2f} A)')
        pull_distances = [float(i) + pull_initial for i in pull_string.split()]

    if 'G6' in complx:
        guest_anchors = [':2@C6', ':2@C1']
        traj = pt.load(os.path.join(complx, 'AMBER', 'solvate.rst7'), 
                            os.path.join(complx, 'AMBER', 'solvate.prmtop'))
        
        pull_initial = pt.distance(traj, ' '.join([dummy_anchors[0], guest_anchors[0]]))[0]
        
        print(f'Set pull offset ({pull_initial:1.2f} A)')
        pull_distances = [float(i) + pull_initial for i in pull_string.split()]
        
    # Set the guest restraint atoms
    ###############################
    print('Set guest restraint atoms...')
    guest_restraint_atoms = [[dummy_anchors[0], guest_anchors[0]],
                             [dummy_anchors[1], dummy_anchors[0], guest_anchors[0]],
                             [dummy_anchors[0], guest_anchors[0], guest_anchors[1]],
                            ]

    guest_restraint_targets = [pull_initial,
                               180.0,
                               180.0
                              ]
    guest_restraint_target_final = [pull_distances[-1], 180.0, 180.0]
    guest_restraint_distance_fc = 5.0 # kcal/mol-A**2
    guest_restraint_angle_fc = 100.0  # kcal/mol-rad**2
    
    hg = pmd.load_file(os.path.join(complx, 'AMBER', 'solvate.prmtop'),
                       os.path.join(complx, 'AMBER', 'solvate.rst7'),
                       structure=True)

    # Set the static restraint atoms
    ################################
    print('Set static restraints...')
    static_restraints = []
    for index, atoms in enumerate(static_restraint_atoms):    
        this = static_DAT_restraint(restraint_mask_list=atoms,
                            num_window_list=windows, 
                            ref_structure=hg,
                            force_constant=static_restraint_angle_fc if len(atoms) > 2 else static_restraint_distance_fc,
                            amber_index=True)
                                    
        static_restraints.append(this)

    # Initialize the restraints
    ###########################
    print('Set guest restraints...')
    guest_restraints = []
    for index, atoms in enumerate(guest_restraint_atoms): 
        if len(atoms) > 2:
            angle = True
        else:
            angle = False
        this = DAT_restraint()
        this.auto_apr = True
        this.amber_index = True
        this.topology = hg
        this.mask1 = atoms[0]
        this.mask2 = atoms[1]
        if angle:
            this.mask3 = atoms[2]
            this.attach['fc_final'] = guest_restraint_angle_fc
        else:
            this.attach['fc_final'] = guest_restraint_distance_fc
        this.attach['target'] = guest_restraint_targets[index]
        this.attach['fraction_list'] = attach_fractions

        this.pull['target_final'] = guest_restraint_target_final[index]
        this.pull['num_windows'] = windows[1]
        this.initialize()
        
        guest_restraints.append(this)
    
    # Create the windows
    ####################
    print('Creating windows...')
    window_list = create_window_list(guest_restraints)
    make_window_dirs(window_list, path=os.path.join(complx, 'AMBER', 'APR'))
    
    # Write the restraint file in each window
    #########################################
    print('Writing restratint file in each window...')
    for window in window_list:
        with open(os.path.join(complx, 'AMBER', 'APR', 'windows', window, 'disang.rest'), 
                  'w') as file:
            for restraint in static_restraints + guest_restraints:
                if window[0] == 'a':
                    phase = 'attach'
                if window[0] == 'p':
                    phase = 'pull'
                if window[0] == 'r':
                    phase = 'release'

                string = amber_restraint_line(restraint, phase, int(window[1:]))
                if string is not None:
                    file.write(string)

./OA-G3-3
Set pull offset (7.46 A)
Set guest restraint atoms...
Set static restraints...


2018-04-23 07:04:11 PM Calculating attach targets and force constants...
2018-04-23 07:04:11 PM Attach, Method #1
2018-04-23 07:04:11 PM Calculating pull targets and force constants...
2018-04-23 07:04:11 PM Pull, Method #1
2018-04-23 07:04:11 PM Calculating release targets and force constants...
2018-04-23 07:04:11 PM No restraint info set for the release phase! Skipping...
2018-04-23 07:04:11 PM Number of attach windows = 15
2018-04-23 07:04:11 PM Number of pull windows = 46
2018-04-23 07:04:11 PM This restraint will be skipped in the release phase
2018-04-23 07:04:11 PM Assigning atom indices...
2018-04-23 07:04:11 PM There are 1 atoms in the mask :3  ...
2018-04-23 07:04:11 PM There are 1 atoms in the mask :1@C26  ...
2018-04-23 07:04:12 PM Calculating attach targets and force constants...
2018-04-23 07:04:12 PM Attach, Method #1
2018-04-23 07:04:12 PM Calculating pull targets and force constants...
2018-04-23 07:04:12 PM Pull, Method #1
2018-04-23 07:04:12 PM Calculating release t

Set guest restraints...


2018-04-23 07:04:14 PM All restraints are "continuous_apr" style.
2018-04-23 07:04:14 PM Restraints appear to be consistent


Creating windows...
Writing restratint file in each window...
./OA-G6-3
Set pull offset (9.57 A)
Set guest restraint atoms...
Set static restraints...


...

Creating windows...
Writing restratint file in each window...
./OA-G3-2
Set pull offset (7.28 A)
Set guest restraint atoms...
Set static restraints...


2018-04-23 07:04:42 PM Calculating attach targets and force constants...
2018-04-23 07:04:42 PM Attach, Method #1
2018-04-23 07:04:42 PM Calculating pull targets and force constants...
2018-04-23 07:04:42 PM Pull, Method #1
2018-04-23 07:04:42 PM Calculating release targets and force constants...
2018-04-23 07:04:42 PM No restraint info set for the release phase! Skipping...
2018-04-23 07:04:42 PM Number of attach windows = 15
2018-04-23 07:04:42 PM Number of pull windows = 46
2018-04-23 07:04:42 PM This restraint will be skipped in the release phase
2018-04-23 07:04:42 PM Assigning atom indices...
2018-04-23 07:04:42 PM There are 1 atoms in the mask :3  ...
2018-04-23 07:04:42 PM There are 1 atoms in the mask :1@C26  ...
2018-04-23 07:04:42 PM Calculating attach targets and force constants...
2018-04-23 07:04:42 PM Attach, Method #1
2018-04-23 07:04:42 PM Calculating pull targets and force constants...
2018-04-23 07:04:42 PM Pull, Method #1
2018-04-23 07:04:42 PM Calculating release t

Set guest restraints...


2018-04-23 07:04:44 PM All restraints are "continuous_apr" style.
2018-04-23 07:04:44 PM Restraints appear to be consistent


Creating windows...
Writing restratint file in each window...


Now, let's copy the structure 

In [7]:
files = ['complex_dummy.pdb',
        'dm1.mol2',
        'dm2.mol2',
        'dm3.mol2',
        'dummy.frcmod',
        'hst.mol2',
        'hst.frcmod',
        'gst.mol2',
        'gst.frcmod']

for complx in complexes:
    for window_index, window in enumerate(window_list):

        os.remove(os.path.join(complx, 'AMBER', 'APR', 'windows', window, 'complex_dummy.pdb'))
        for file in files:
            try:
                os.symlink(os.path.join('../../../', file),
                           os.path.join(complx, 'AMBER', 'APR', 'windows', window, file))        
            except:
                pass

In [8]:
for complx in complexes:
    attach_windows = len(attach_fractions) - 1
    pull_windows = len(pull_distances)
    
    for window in range(attach_windows):
        shutil.copy(os.path.join(complx, 'AMBER', 'APR', 'windows', f'a{window:03d}', 'complex_dummy.pdb'),
                   os.path.join(complx, 'AMBER', 'APR', 'windows', f'a{window:03d}', 'hg.pdb'))
    
    for window in range(pull_windows):
        hg = pmd.load_file(os.path.join(complx, 'AMBER', 'APR', 'windows', f'p{window:03d}', 'complex_dummy.pdb'))
        
        target_difference = guest_restraints[0].phase['pull']['targets'][window] - \
                            guest_restraints[0].pull['target_initial']
        
        print(f'{complx}: Translating p{window:03d} by {target_difference:03f} A...')
        for atom in hg.atoms:
            if atom.residue.name == 'GST':
                atom.xz += target_difference
        hg.save(os.path.join(complx, 'AMBER', 'APR', 'windows', f'p{window:03d}', 'hg.pdb'),
               overwrite=True)

./OA-G3-3: Translating p000 by 0.000000 A...
./OA-G3-3: Translating p001 by 0.400000 A...
./OA-G3-3: Translating p002 by 0.800000 A...
./OA-G3-3: Translating p003 by 1.200000 A...
./OA-G3-3: Translating p004 by 1.600000 A...
./OA-G3-3: Translating p005 by 2.000000 A...
./OA-G3-3: Translating p006 by 2.400000 A...
./OA-G3-3: Translating p007 by 2.800000 A...
./OA-G3-3: Translating p008 by 3.200000 A...
./OA-G3-3: Translating p009 by 3.600000 A...
./OA-G3-3: Translating p010 by 4.000000 A...
./OA-G3-3: Translating p011 by 4.400000 A...
./OA-G3-3: Translating p012 by 4.800000 A...
./OA-G3-3: Translating p013 by 5.200000 A...
./OA-G3-3: Translating p014 by 5.600000 A...
./OA-G3-3: Translating p015 by 6.000000 A...
./OA-G3-3: Translating p016 by 6.400000 A...
./OA-G3-3: Translating p017 by 6.800000 A...
./OA-G3-3: Translating p018 by 7.200000 A...
./OA-G3-3: Translating p019 by 7.600000 A...
./OA-G3-3: Translating p020 by 8.000000 A...
./OA-G3-3: Translating p021 by 8.400000 A...
./OA-G3-3:

In [22]:
import logging
from importlib import reload
reload(logging)

logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%Y-%m-%d %I:%M:%S %p',
                    level=logging.INFO)
logging.info('Started logging...')

2018-04-23 07:55:07 PM Started logging...


In [23]:
for complx in complexes:
    for window_index, window in enumerate(window_list):
        print(complx, window)
        
        system = System()

        system.output_path = os.path.join(complx, 'AMBER', 'APR', 'windows', window)        

        system.target_waters = 3000
        system.output_prefix = 'solvate'

        system.neutralize = True
        system.pbc_type = 'rectangular'
        system.add_ions = ['Na+', 4, 'Cl-', 4]
        system.template_lines = ['source leaprc.water.tip3p',
                               'source leaprc.gaff',
                               'loadamberparams hst.frcmod',
                               'HST = loadmol2 hst.mol2',
                               'loadamberparams gst.frcmod',
                               'GST = loadmol2 gst.mol2',
                               'loadamberparams dummy.frcmod',
                               'DM1 = loadmol2 dm1.mol2',
                               'DM2 = loadmol2 dm2.mol2',
                               'DM3 = loadmol2 dm3.mol2',
                               'model = loadpdb hg.pdb'
        ]

        try:
            system.build()
        except:
            system.build()

./OA-G3-3 a000


2018-04-23 07:55:11 PM Cl-                 4
2018-04-23 07:55:11 PM DM1                 1
2018-04-23 07:55:11 PM DM2                 1
2018-04-23 07:55:11 PM DM3                 1
2018-04-23 07:55:11 PM GST                 1
2018-04-23 07:55:11 PM HST                 1
2018-04-23 07:55:11 PM Na+                13
2018-04-23 07:55:11 PM WAT              3000


./OA-G3-3 a001


2018-04-23 07:55:15 PM Cl-                 4
2018-04-23 07:55:15 PM DM1                 1
2018-04-23 07:55:15 PM DM2                 1
2018-04-23 07:55:15 PM DM3                 1
2018-04-23 07:55:15 PM GST                 1
2018-04-23 07:55:15 PM HST                 1
2018-04-23 07:55:15 PM Na+                13
2018-04-23 07:55:15 PM WAT              3001
2018-04-23 07:55:15 PM The final solvation step added the wrong number of waters. Repeating ...
2018-04-23 07:55:15 PM Cl-                 4
2018-04-23 07:55:15 PM DM1                 1
2018-04-23 07:55:15 PM DM2                 1
2018-04-23 07:55:15 PM DM3                 1
2018-04-23 07:55:15 PM GST                 1
2018-04-23 07:55:15 PM HST                 1
2018-04-23 07:55:15 PM Na+                13
2018-04-23 07:55:15 PM WAT              3001
2018-04-23 07:55:15 PM The final solvation step added the wrong number of waters. Repeating ...
2018-04-23 07:55:15 PM Cl-                 4
2018-04-23 07:55:15 PM DM1                 

./OA-G3-3 a002


...

2018-04-23 08:26:01 PM Cl-                 4
2018-04-23 08:26:01 PM DM1                 1
2018-04-23 08:26:01 PM DM2                 1
2018-04-23 08:26:01 PM DM3                 1
2018-04-23 08:26:01 PM GST                 1
2018-04-23 08:26:01 PM HST                 1
2018-04-23 08:26:01 PM Na+                13
2018-04-23 08:26:01 PM WAT              3000


./OA-G3-2 p044


2018-04-23 08:26:04 PM Cl-                 4
2018-04-23 08:26:04 PM DM1                 1
2018-04-23 08:26:04 PM DM2                 1
2018-04-23 08:26:04 PM DM3                 1
2018-04-23 08:26:04 PM GST                 1
2018-04-23 08:26:04 PM HST                 1
2018-04-23 08:26:04 PM Na+                13
2018-04-23 08:26:04 PM WAT              3000


./OA-G3-2 p045


2018-04-23 08:26:06 PM Cl-                 4
2018-04-23 08:26:06 PM DM1                 1
2018-04-23 08:26:06 PM DM2                 1
2018-04-23 08:26:06 PM DM3                 1
2018-04-23 08:26:06 PM GST                 1
2018-04-23 08:26:06 PM HST                 1
2018-04-23 08:26:06 PM Na+                13
2018-04-23 08:26:06 PM WAT              3000
